<a href="https://colab.research.google.com/github/DenisVasil/LLM_examples/blob/main/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat Models (čata modeļi)

In [1]:
!pip install langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.9 MB/s eta 0:00:00


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=GOOGLE_API_KEY
)


result = model.invoke("What is 81 divided by 9")

print("Full result:")
print(result)
print("Context:")
print(result.content)

Full result:
content='81 divided by 9 is 9.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-24d7f7c1-7122-4f1b-9d5e-6bcd40e69444-0' usage_metadata={'input_tokens': 9, 'output_tokens': 11, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}}
Context:
81 divided by 9 is 9.


In [3]:
# Basic Conversation

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=GOOGLE_API_KEY
)

messages = [
    SystemMessage(content="Solve the following math problem"),
    HumanMessage(content="What is 81 divided by 9?")
]

result = model.invoke(messages)

print(f"Answer from AI: {result.content}")

messages = [
    SystemMessage(content="Solve the following math problem"),
    HumanMessage(content="What is 81 divided by 9?"),
    AIMessage(content=f"{result.content}"),
    HumanMessage(content="Divide this result by 3")
]

result = model.invoke(messages)
print(f"New answer from AI: {result.content}")


Answer from AI: 81 divided by 9 is 9.
New answer from AI: 9 divided by 3 is 3.


In [ ]:
from langchain.memory.chat_memory import InMemoryChatMessageHistory

history = InMemoryChatMessageHistory()
history.add_message({"role": "user", "content": "Hello"})
history.add_message({"role": "assistant", "content": "Hi there!"})

print(history.messages)

[{'role': 'user', 'content': 'Hello'}, {'role': 'assistant', 'content': 'Hi there!'}]


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage # Import necessary classes

memory = ConversationBufferMemory()
memory.chat_memory.add_message(HumanMessage(content="Hello")) # Use HumanMessage
memory.chat_memory.add_message(AIMessage(content="Hi there!")) # Use AIMessage

print(memory.load_memory_variables({}))

{'history': 'Human: Hello\nAI: Hi there!'}


<ipython-input-6-27d192231410>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [ ]:
# chat
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=GOOGLE_API_KEY
)


chat_history = []

system_message = SystemMessage(content="You are helpful AI assistant")
chat_history.append(system_message)

# chat loop
while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    chat_history.append(HumanMessage(content=query))

    result = model.invoke(chat_history)
    response = result.content
    chat_history.append(AIMessage(content=response))
    print(f"AI response: {response}")

print("----- Message history ------")
print(chat_history)

You: How many birds are there in Latvia?
AI response: It's impossible to give an exact number of birds in Latvia at any given moment.  Bird populations are dynamic, constantly changing due to:

* **Migration:** Latvia is on a major migration route, so the number of birds present fluctuates dramatically throughout the year.  Millions of birds pass through during spring and fall migrations.
* **Breeding Cycles:** Bird numbers increase during breeding seasons as chicks hatch and fledge.
* **Food Availability:** Bird populations are affected by the availability of food resources, which can vary seasonally and annually.
* **Weather Conditions:** Harsh winters or extreme weather events can impact bird survival rates.
* **Habitat Changes:** Loss of habitat due to human activities affects bird populations.

While an exact count isn't feasible, ornithologists and researchers in Latvia *can* estimate populations of specific species through various methods like bird counts, ringing, and nest moni

In [ ]:
!pip install duckdb

In [ ]:
# Saving Chat History
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from google.colab import userdata

from langchain.memory.chat_memory import InMemoryChatMessageHistory
from langchain.schema import HumanMessage, AIMessage
import duckdb
import os

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=GOOGLE_API_KEY
)


# Constants
DB_PATH = "chat_history.db"
TABLE_NAME = "chat_history"
SESSION_ID = "user_session_new"  # This could be a username or unique ID

# Initialize DuckDB
con = duckdb.connect(DB_PATH)
con.execute(f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
    session_id TEXT,
    role TEXT,
    content TEXT
)
""")

# Helper Functions


def save_message_to_db(session_id, role, content):
    """Save a single message to the DuckDB database."""
    con.execute(f"""
    INSERT INTO {TABLE_NAME} (session_id, role, content)
    VALUES (?, ?, ?)
    """, [session_id, role, content])


def load_messages_from_db(session_id):
    """Load chat history for a specific session from the DuckDB database."""
    result = con.execute(f"""
    SELECT role, content FROM {TABLE_NAME} WHERE session_id = ? ORDER BY rowid
    """, [session_id]).fetchall()
    return [HumanMessage(content=row[1]) if row[0] == "user" else AIMessage(content=row[1]) for row in result]


# Initialize Chat Message History
print("Initializing DuckDB Chat Message History...")
messages = load_messages_from_db(SESSION_ID)
chat_history = InMemoryChatMessageHistory(messages=messages)

print("Chat History Initialized.")
print("Current Chat History:", [msg.content for msg in chat_history.messages])


print("Start chatting with the AI. Type 'exit' to quit.")

while True:
    human_input = input("User: ")
    if human_input.lower() == "exit":
        break

    # Save and process user message
    chat_history.add_user_message(human_input)
    save_message_to_db(SESSION_ID, "user", human_input)

    # Generate AI response
    ai_response = model.invoke(chat_history.messages)
    chat_history.add_ai_message(ai_response.content)
    save_message_to_db(SESSION_ID, "ai", ai_response.content)

    print(f"AI: {ai_response.content}")

# Close DuckDB connection when done
con.close()

Initializing DuckDB Chat Message History...
Chat History Initialized.
Current Chat History: []
Start chatting with the AI. Type 'exit' to quit.
User: How many bird species are there in Latvia?
AI: While the total number of bird species *ever recorded* in Latvia is over 360, the number of *regularly occurring* species is closer to **around 300**.  This includes breeding species, regular migrants, and winter visitors.  The exact number fluctuates slightly based on taxonomic changes and ongoing research.
User: Which species are the most common?
AI: Some of the most common bird species in Latvia include:

* **Great Tit (Parus major):**  A widespread and adaptable species found in various habitats, including forests, parks, and gardens.
* **Willow Warbler (Phylloscopus trochilus):** A common summer migrant, filling woodlands with its distinctive song.
* **Chaffinch (Fringilla coelebs):** Another abundant finch, often seen in gardens and woodlands.
* **Common Starling (Sturnus vulgaris):**  

In [ ]:
!pip install -U langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from google.colab import userdata
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=GOOGLE_API_KEY
)

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! It's nice to meet you. How can I help you today?


In [ ]:
app = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You told me your name is Bob.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from google.colab import userdata
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=GOOGLE_API_KEY
)

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

while True:
    query = input("You: ")
    if query.lower() == "exit":
        break

    input_messages = [HumanMessage(query)]
    output = app.invoke({"messages": input_messages}, config)
    output["messages"][-1].pretty_print()  # output contains all messages in state


You: How many fish species are there in Baltic Sea?
================================== Ai Message ==================================

There are roughly **170** fish species found in the Baltic Sea, although the exact number can fluctuate slightly depending on the criteria used (e.g., whether to include occasional visitors).  It's important to note that the Baltic is a brackish environment, meaning fewer species tolerate its lower salinity compared to the fully saline North Sea.
You: Which are the most commoon?
================================== Ai Message ==================================

The most common fish species in the Baltic Sea include:

* **Herring:**  Arguably the most abundant fish, crucial both ecologically and commercially.  Several different herring stocks exist within the Baltic.
* **Sprat:** Another small, oily fish, similar to herring and also commercially important.
* **Cod:**  A key predator and commercially valuable, though Baltic cod populations have faced challen

# Prompt Templates (Veidnes Uzvednēm)

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

prompt_template.invoke({"topic": "cats"})

StringPromptValue(text='Tell me a joke about cats')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})
# prompt = prompt_template.invoke({"topic": "cats"})
# result = model.invoke(prompt)
# print(result.content)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

prompt_template.invoke({"msgs": [HumanMessage(content="hi!"), HumanMessage(content="How are you?")]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}), HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={})])

In [ ]:
# AIMessage, HumanMessage, SystemMessage

from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=GOOGLE_API_KEY
)


template = "Tell me a joke about {topic}"
prompt_template = ChatPromptTemplate.from_template(template)

print("------Prompt from template------")
prompt = prompt_template.invoke({"topic": "cats"})
print(prompt)

result = model.invoke(prompt)
print("\n------Result-------")
print(result.content)


------Prompt from template------
messages=[HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})]

------Result-------
Why was the cat sitting on the computer? 

To keep an eye on the mouse!


In [ ]:
# Create a ChatPromptTemplate using a template string
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

template = "Tell me a joke about {topic}."
prompt_template = ChatPromptTemplate.from_template(template)

print("-----Prompt from Template-----")
prompt = prompt_template.invoke({"topic": "cats"})
print(prompt)

-----Prompt from Template-----
messages=[HumanMessage(content='Tell me a joke about cats.', additional_kwargs={}, response_metadata={})]


In [ ]:
# Prompt with Multiple Placeholders

from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

template_multiple = """You are a helpful assistant.
Human: Tell me a {adjective} story about a {animal}.
Assistant:"""
prompt_multiple = ChatPromptTemplate.from_template(template_multiple)
prompt = prompt_multiple.invoke({"adjective": "funny", "animal": "panda"})
print("\n----- Prompt with Multiple Placeholders -----\n")
print(prompt)


----- Prompt with Multiple Placeholders -----

messages=[HumanMessage(content='You are a helpful assistant.\nHuman: Tell me a funny story about a panda.\nAssistant:', additional_kwargs={}, response_metadata={})]


In [ ]:
# Prompt with System and Human Messages (Using Tuples)
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human", "Tell me {joke_count} jokes."),
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic": "lawyers", "joke_count": 3})
print("\n----- Prompt with System and Human Messages (Tuple) -----\n")
print(prompt)


----- Prompt with System and Human Messages (Tuple) -----

messages=[SystemMessage(content='You are a comedian who tells jokes about lawyers.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me 3 jokes.', additional_kwargs={}, response_metadata={})]


# Chains (Ķēdes)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=GOOGLE_API_KEY
)

prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me a {adjective} joke")],
)

chain = prompt | model | StrOutputParser()

chain.invoke({"adjective": "funny"})

"Why don't scientists trust atoms? \n\nBecause they make up everything!"

In [ ]:
# Basic Chain

from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.messages import HumanMessage

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}."),
        ("human", "Tell me {joke_count} jokes."),
    ]
)
# StrOutputParser() ~ .content()
# LangChain expression language
chain = prompt_template | model | StrOutputParser()

result = chain.invoke({"topic": "fish", "joke_count": 3})

print(result)


1. Why did the fish blush? Because it saw the ocean floor!  (Pause for groans, then add)  It was a coral-reef-ly embarrassing moment.


2. What do you call a fish with no eyes?  Fsh! (Say it really fast).


3.  I went to a seafood disco last week...  It was legen-dairy!  (Pause) ...and there was a killer whale playing the decks.  He was *really* good at spinning the tunes.


In [ ]:
# Extended Chain

from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_core.messages import HumanMessage

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}."),
        ("human", "Tell me {joke_count} jokes."),
    ]
)

uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Word count {len(x.split())} \n {x}")

chain = prompt_template | model | StrOutputParser() | uppercase_output | count_words

result = chain.invoke({"topic": "fish", "joke_count": 3})

print(result)


Word count 58 
 1. WHY DID THE FISH BLUSH?  BECAUSE IT SAW THE CLAM!  (PAUSE FOR GROANS, THEN ADD)  ...AND IT WAS SHELLFISH ABOUT IT.


2. WHAT DO YOU CALL A FISH WITH NO EYES?  FSH!


3.  I WENT TO A SEAFOOD DISCO LAST WEEK...  IT WAS ALRIGHT, BUT THERE WASN'T MUCH COD DANCING.  AND THE BASS WAS A BIT REPETITIVE.


In [ ]:
# Parallel
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY
)


# Define prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert product reviewer."),
        ("human", "List the main features of the product {product_name}."),
    ]
)


# Define pros analysis step
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the pros of these features.",
            ),
        ]
    )
    return pros_template.format_prompt(features=features)


# Define cons analysis step
def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the cons of these features.",
            ),
        ]
    )
    return cons_template.format_prompt(features=features)


# Combine pros and cons into a final review
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"


# Simplify branches with LCEL
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

# Create the combined chain using LangChain Expression Language (LCEL)
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

# Run the chain
result = chain.invoke({"product_name": "MacBook Pro"})

# Output
print(result)


Pros:
You're right, providing a comprehensive review of a MacBook Pro without specifying the model is like reviewing "a car" – too broad to be useful.  The variations between models are substantial, impacting performance and value significantly.

However, based on your provided core features, let's outline the *general* pros of those features, acknowledging that the degree to which these pros manifest will vary considerably depending on the specific model:


**Pros of Core MacBook Pro Features:**

* **High-Performance Processors (M-series or Intel):**
    * **Pro:**  Seamless multitasking, rapid application launch times, effortless handling of demanding applications (video editing, 3D rendering, etc.).  This translates to increased productivity and a more fluid user experience.  The M-series chips offer particularly impressive performance and energy efficiency.

* **Powerful Graphics (Integrated or Dedicated):**
    * **Pro:**  Smooth video playback, excellent performance in graphicall

In [ ]:
# Branching
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY
)

# Define prompt templates for different feedback types
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a response addressing this negative feedback: {feedback}."),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

# Define the feedback classification template
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."),
    ]
)

# Define the runnable branches for handling feedback
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | model | StrOutputParser()  # Positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | model | StrOutputParser()  # Negative feedback chain
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | model | StrOutputParser()  # Neutral feedback chain
    ),
    escalate_feedback_template | model | StrOutputParser()
)

# Create the classification chain
classification_chain = classification_template | model | StrOutputParser()

# Combine classification and response generation into one chain
chain = classification_chain | branches

# Run the chain with an example review
# Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
# Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
# Neutral review - "The product is okay. It works as expected but nothing exceptional."
# Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"

review = "The product is okay. It works as expected but nothing exceptional."
result = chain.invoke({"feedback": review})

# Output the result
print(result)

Several options, depending on your desired tone and relationship with the feedback provider:


**Option 1 (Formal & Brief):**

> Thank you for your feedback. We appreciate you taking the time to share your thoughts on [project/product/service]. We will take your comments into consideration as we move forward.


**Option 2 (Slightly More Engaging):**

> Thank you for your feedback on [project/product/service]. We value your input and appreciate you letting us know your experience.  We're always striving to improve, and your comments will help us in that process.


**Option 3 (If you want to gently probe for more detail, suitable only if you have a good relationship):**

> Thank you for your feedback on [project/product/service].  We appreciate you taking the time to share your thoughts.  While we're pleased to hear it met your expectations, we'd be grateful if you could elaborate on any areas where we could potentially improve or exceed expectations in the future.


**Option 4 (If you s

# Document Loaders (Dokumentu Ielādētāji)

In [4]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

file_path = "/content/diabetes.csv"

loader = CSVLoader(file_path=file_path)
data = loader.load()

for record in data[:2]:
    #print(type(record))
    print(record)

page_content='Pregnancies: 6
Glucose: 148
BloodPressure: 72
SkinThickness: 35
Insulin: 0
BMI: 33.6
DiabetesPedigreeFunction: 0.627
Age: 50
Outcome: 1' metadata={'source': '/content/diabetes.csv', 'row': 0}
page_content='Pregnancies: 1
Glucose: 85
BloodPressure: 66
SkinThickness: 29
Insulin: 0
BMI: 26.6
DiabetesPedigreeFunction: 0.351
Age: 31
Outcome: 0' metadata={'source': '/content/diabetes.csv', 'row': 1}


In [5]:
!pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 8.0 MB/s eta 0:00:00


In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/invoice_1001329.pdf")
pages = []

async for page in loader.alazy_load():
    pages.append(page)
print(f"{pages[0].metadata}\n")
print(pages[0].page_content)

{'producer': 'Qt 4.8.6', 'creator': 'wkhtmltopdf 0.12.2', 'creationdate': '2023-05-05T01:13:00+00:00', 'title': 'Invoice Preview', 'source': '/content/invoice_1001329.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}

ExcelCult
India
Bill To
Ms.Santoshi
Mumbai, India
Santoshvarma0988@gmail.com
9999999999
 
  
Invoice no.
1001329
Date
5/4/2023
Description
Quantity
Unit price
Amount
Total
$2,200.00
Office Chair
2
$1,100.00
$2,200.00
System Generated


# Agents (Aģenti)

In [ ]:
!pip install -qU duckduckgo-search langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.7 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

from langchain.agents import initialize_agent, Tool, AgentType
from langchain.agents import AgentExecutor
from langchain_community.tools import DuckDuckGoSearchRun


GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY
)

# Define the tool: DuckDuckGo search results
ddg_search = DuckDuckGoSearchRun()

# Set up the agent with the search tool
tools = [Tool(
    name="DuckDuckGoSearchRun",
    func=ddg_search.run,
    description="Searches the web using DuckDuckGo."
)]

# Define a simple prompt template for asking a question
prompt_template = "Search the web for information about {query}"

# Initialize the agent with a basic agent type
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Use the agent to interact and fetch search results
def get_search_results_using_agent(query: str):
    query_formatted = prompt_template.format(query=query)
    response = agent.run(query_formatted)
    return response

# Example usage
query = "latest news on AI advancements"
search_results = get_search_results_using_agent(query)
print(search_results)


<ipython-input-35-f7b5cff39ffa>:32: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
<ipython-input-35-f7b5cff39ffa>:42: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query_formatted)




> Entering new AgentExecutor chain...
Thought: I need to use DuckDuckGoSearchRun to find the latest news on AI advancements.  I'll use a broad search term to get a range of results.

Action: DuckDuckGoSearchRun
Action Input: "latest AI advancements news"
Observation: Google shared its latest AI news from January 2025, highlighting advancements in products, research, and more. Gemini 2.0 Flash, a performance upgrade to the Gemini app, delivers faster responses and more capable help. Jan. 2, 2025 — Artificial intelligence has the potential to improve the analysis of medical image data. For example, algorithms based on deep learning can determine the location and size of tumors. Discover the latest AI advancements. From IBM's and Meta's new tools to groundbreaking research and new AI laws, our recent AI news keeps you informed on all fronts. Significant AI developments in August 2024 include innovations in artificial intelligence, regulatory discussions about AI, and notable releases. K

In [ ]:
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

from langchain.agents import initialize_agent, Tool, AgentType
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer


GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY
)


from sklearn.datasets import load_iris

def load_data(query=None):  # Adding query parameter to make it compatible
    # Load Iris dataset from sklearn
    iris = load_iris()
    df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    df['species'] = iris.target
    return df

# Function to clean data (simple imputation for missing values)
def clean_data(df: pd.DataFrame, query=None):
    imputer = SimpleImputer(strategy="mean")
    df_clean = df.copy()
    df_clean[:] = imputer.fit_transform(df)
    return df_clean

# Function to train a model (Random Forest)
def train_model(df: pd.DataFrame, target_column: str, query=None):
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a Random Forest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Set up tools for the agent
tools = [
    Tool(
        name="Load Data",
        func=load_data,
        description="Loads the Iris dataset."
    ),
    Tool(
        name="Clean Data",
        func=clean_data,
        description="Cleans the dataset by handling missing values."
    ),
    Tool(
        name="Train Model",
        func=train_model,
        description="Trains a Random Forest model and returns the accuracy."
    )
]

# Initialize the agent with tools and LLM
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Automating a data science task
def automate_data_science_task():
    # Step 1: Load data
    print("Loading data...")
    df = load_data()

    # Step 2: Clean data
    print("Cleaning data...")
    df_clean = clean_data(df)

    # Step 3: Train model
    print("Training model...")
    accuracy = train_model(df_clean, target_column="species")

    return accuracy

# Run the agent
accuracy = automate_data_science_task()
print(f"Model Accuracy: {accuracy}")

Loading data...
Cleaning data...
Training model...
Model Accuracy: 1.0


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

from langchain_google_genai import ChatGoogleGenerativeAI

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize Chat Model

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=GOOGLE_API_KEY
)


# Create a tool
@tool
def super_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2


tools = [super_function]


query = "what is the value of super_function(3)?"

langgraph_agent_executor = create_react_agent(model, tools)


messages = langgraph_agent_executor.invoke({"messages": [("human", query)]})
{
    "input": query,
    "output": messages["messages"][-1].content,
}

{'input': 'what is the value of super_function(3)?',
 'output': "The value of `super_function(3)` is `{'output': 5}`."}

# Tools (Rīki)

In [ ]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [ ]:
!pip install langchain-experimental

In [ ]:
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
python_repl.run("print(1+1)")


'2\n'

# Invoice extractor (Rēķinu izvilkšanas lietotne)

In [ ]:
!pip install pypdf

In [7]:
from pypdf import PdfReader
pdf_doc = "/content/invoice_1001329.pdf"
pdf_reader = PdfReader(pdf_doc)
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()
print(text)

ExcelCult
India
Bill To
Ms.Santoshi
Mumbai, India
Santoshvarma0988@gmail.com
9999999999
 
  
Invoice no.
1001329
Date
5/4/2023
Description
Quantity
Unit price
Amount
Total
$2,200.00
Office Chair
2
$1,100.00
$2,200.00
System Generated
 


In [8]:
import pandas as pd
import re
from pydantic import BaseModel, ValidationError
from pypdf import PdfReader
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from langchain.schema import HumanMessage

# Set up your Google API key
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found in userdata.")

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7,
)

# Define a Pydantic model for invoice data validation
class InvoiceData(BaseModel):
    Invoice_no: str
    Description: str
    Quantity: str
    Date: str
    Unit_price: str
    Amount: float
    Total: float
    Email: str
    Phone_number: str
    Address: str

# PDF file path
pdf_path = "/content/invoice_1001329.pdf"

# Extract text from PDF
print(f"Processing {pdf_path}...")
text = ""
pdf_reader = PdfReader(pdf_path)
for page in pdf_reader.pages:
    text += page.extract_text()

# Clean and prepare the text
text = text.replace("\n", " ").strip()

# Construct the prompt
prompt = f"""Extract all the following values: invoice no., Description, Quantity, date,
Unit price, Amount, Total, email, phone number, and address from this data:

{text}

Expected output (JSON format):
{{
    "Invoice_no": "1001329",
    "Description": "Office Chair",
    "Quantity": "2",
    "Date": "5/4/2023",
    "Unit_price": "1100.00",
    "Amount": 2200.00,
    "Total": 2200.00,
    "Email": "Santoshvarma0988@gmail.com",
    "Phone_number": "9999999999",
    "Address": "Mumbai, India"
}}
"""

# Debug: Print prompt to verify it
print("Prompt sent to LLM:\n", prompt)

# Initialize a DataFrame
df = pd.DataFrame(columns=[
    'Invoice_no', 'Description', 'Quantity', 'Date', 'Unit_price',
    'Amount', 'Total', 'Email', 'Phone_number', 'Address'
])

# Generate response from LLM
try:
    llm_response = llm.invoke([HumanMessage(content=prompt)])  # Use `invoke()` instead of calling the model directly
    llm_content = llm_response.content  # Access response content

    # Clean response: Remove triple backticks and JSON keyword
    cleaned_text = re.sub(r"```json\s*|\s*```", "", llm_content).strip()

    # Validate and parse JSON response using Pydantic
    invoice_data = InvoiceData.model_validate_json(cleaned_text)
    print("Validated Data:\n", invoice_data)

    # Convert Pydantic model to dict and append to DataFrame
    new_data = pd.DataFrame([invoice_data.model_dump()])

    # Append new data safely
    if df.empty:
        df = new_data
    else:
        df = pd.concat([df, new_data], ignore_index=True)

except ValidationError as e:
    print("Validation Error:\n", e)
except Exception as e:
    print(f"Error generating LLM response: {e}")

# Save to CSV
csv_filename = 'invoices_summary.csv'
df.to_csv(csv_filename, index=False)
print(f"CSV file created as '{csv_filename}'.")


Processing /content/invoice_1001329.pdf...
Prompt sent to LLM:
 Extract all the following values: invoice no., Description, Quantity, date,
Unit price, Amount, Total, email, phone number, and address from this data:

ExcelCult India Bill To Ms.Santoshi Mumbai, India Santoshvarma0988@gmail.com 9999999999      Invoice no. 1001329 Date 5/4/2023 Description Quantity Unit price Amount Total $2,200.00 Office Chair 2 $1,100.00 $2,200.00 System Generated

Expected output (JSON format):
{
    "Invoice_no": "1001329",
    "Description": "Office Chair",
    "Quantity": "2",
    "Date": "5/4/2023",
    "Unit_price": "1100.00",
    "Amount": 2200.00,
    "Total": 2200.00,
    "Email": "Santoshvarma0988@gmail.com",
    "Phone_number": "9999999999",
    "Address": "Mumbai, India"
}

Validated Data:
 Invoice_no='1001329' Description='Office Chair' Quantity='2' Date='5/4/2023' Unit_price='1100.00' Amount=2200.0 Total=2200.0 Email='Santoshvarma0988@gmail.com' Phone_number='9999999999' Address='Mumbai, I

In [10]:
import pandas as pd
import re
import json
from pydantic import BaseModel, field_validator
from typing import Optional
from pypdf import PdfReader
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from langchain.schema import HumanMessage

# Set up Google API key
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found in userdata.")

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7,
)

# Define a Pydantic model with relaxed validation
class InvoiceData(BaseModel):
    Invoice_no: str
    Description: str
    Quantity: str
    Date: str
    Unit_price: str
    Amount: Optional[float] = None
    Total: Optional[float] = None
    Email: Optional[str] = None
    Phone_number: Optional[str] = None
    Address: Optional[str] = None

    # Convert currency values to float
    @field_validator("Amount", "Total", mode="before")
    @classmethod
    def parse_currency(cls, value):
        if isinstance(value, str):
            value = re.sub(r"[^\d.]", "", value)  # Remove non-numeric characters
        return float(value) if value else None

# List of PDF file paths
pdf_files = [
    "/content/invoice_1001329.pdf",
    "/content/invoice_2001321.pdf",
    "/content/invoice_3452334.pdf",
]

# Initialize an empty DataFrame
df = pd.DataFrame(columns=[
    "Invoice_no", "Description", "Quantity", "Date", "Unit_price",
    "Amount", "Total", "Email", "Phone_number", "Address"
])

# Loop through PDF files
for pdf_path in pdf_files:
    try:
        print(f"Processing {pdf_path}...")

        # Extract text from PDF
        text = ""
        pdf_reader = PdfReader(pdf_path)
        for page in pdf_reader.pages:
            text += page.extract_text() or ""  # Ensure it doesn't break if text extraction fails

        text = text.replace("\n", " ").strip()

        # Construct the prompt
        prompt = f"""Extract all the following values: invoice no., Description, Quantity, Date,
        Unit price, Amount, Total, Email, Phone number, and Address from this data:

        {text}

        Expected output (JSON format):
        {{
            "Invoice_no": "1001329",
            "Description": "Office Chair",
            "Quantity": "2",
            "Date": "5/4/2023",
            "Unit_price": "1100.00",
            "Amount": 2200.00,
            "Total": 2200.00,
            "Email": "Santoshvarma0988@gmail.com",
            "Phone_number": "9999999999",
            "Address": "Mumbai, India"
        }}
        """

        # Send prompt to LLM
        llm_response = llm.invoke([HumanMessage(content=prompt)])
        llm_content = llm_response.content

        # Extract JSON using regex
        json_match = re.search(r"\{.*\}", llm_content, re.DOTALL)
        if not json_match:
            print(f"Error: Could not extract JSON from response for {pdf_path}. Skipping.")
            continue

        cleaned_text = json_match.group(0)

        # Ensure valid JSON format
        try:
            invoice_json = json.loads(cleaned_text)
        except json.JSONDecodeError:
            print(f"Error: LLM response is not valid JSON for {pdf_path}: {cleaned_text}")
            continue  # Skip this file

        # Validate and parse JSON response using Pydantic
        try:
            invoice_data = InvoiceData(**invoice_json)
        except Exception as e:
            print(f"Validation Error in {pdf_path}: {e}")
            continue

        print("Validated Data:\n", invoice_data)

        # Convert Pydantic model to dict and append to DataFrame
        new_data = pd.DataFrame([invoice_data.model_dump()])
        df = pd.concat([df, new_data], ignore_index=True)

    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")

# Save to CSV
csv_filename = "multi_invoice_summary.csv"
df.to_csv(csv_filename, index=False)
print(f"CSV file created as '{csv_filename}'.")
print(f"Total invoices processed: {len(df)}")

Processing /content/invoice_1001329.pdf...
Validated Data:
 Invoice_no='1001329' Description='Office Chair' Quantity='2' Date='5/4/2023' Unit_price='1100.00' Amount=2200.0 Total=2200.0 Email='Santoshvarma0988@gmail.com' Phone_number='9999999999' Address='Mumbai, India'
Processing /content/invoice_2001321.pdf...


<ipython-input-10-512997803a29>:121: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_data], ignore_index=True)


Validated Data:
 Invoice_no='2001321' Description='HP Laptop' Quantity='1' Date='5/4/2023' Unit_price='$500.00' Amount=500.0 Total=500.0 Email='sharathkumarraju@proton.me' Phone_number='8888888888' Address='Hyderabad, India'
Processing /content/invoice_3452334.pdf...
Validated Data:
 Invoice_no='3452334' Description='Graphics Card' Quantity='1' Date='5/5/2023' Unit_price='50.00' Amount=50.0 Total=50.0 Email='N/A' Phone_number='N/A' Address='N/A'
CSV file created as 'multi_invoice_summary.csv'.
Total invoices processed: 3
